<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Copy_of_NEWSOLUTION_FINE_TUNING_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries Installation

In [ ]:
!pip install datasets networkx -q

!pip install torch_geometric -q


# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 , L4  IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

!pip install mistral_inference -q

!pip install trl==0.8.6 -q


!pip install torch-geometric -q
!pip install sqlparse networkx -q

!pip install bitsandbytes -q


#!pip uninstall -y torchvision -q
!pip install torchvision --no-cache-dir -q
#import evaluate

!pip install sentence-transformers

## Original Code

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Mistral_7B_Instruct_v0_1_for_text_to_SQL_EVALDATA.ipynb

* Import Main Components

In [ ]:
!pip cache purge

Files removed: 391


In [ ]:
!pip install --upgrade torch torchvision transformers sentence-transformers evaluate -q

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset

from peft import LoraConfig, get_peft_model, TaskType

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)

#from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import spacy
import numpy as np

from torch_geometric.nn import GAT

from trl import setup_chat_format

import colab_env
import evaluate

Mounted at /content/gdrive


In [ ]:
train_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/train_dataset.json", split="train")


In [ ]:
train_dataset[0]['messages'][2]['content']

'SELECT COUNT(total) FROM table_name_92 WHERE finish = "t32"'

In [ ]:
len(train_dataset)

10000

* Original FULL Code - 1

In [ ]:

                # Print valid_mask
                print("Valid mask:", valid_mask)

                # Print shapes and values BEFORE filtering
                print('\n')
                print("Logits shape before filtering:", logits.shape)
                print("Labels shape before filtering:", labels.shape) # Print the correct shape now
                print('\n')
                print("Sample logits before filtering:", logits[0, :10])  # Print first 10 logits of the first sample
                print("Sample labels before filtering:", labels[:10])     # Index labels as a 1D tensor
                print('\n')


* https://stackoverflow.com/questions/70950706/assertionerror-in-torch-geometric-nn-gatconv

Absolutely! Let's describe the dataflow in the provided code, breaking it down step-by-step:

1. **Data Input and Preparation:**
   - The process begins by loading the "sql-create-context" dataset, which presumably contains pairs of natural language questions and their corresponding SQL queries.
   - The dataset is divided into three distinct subsets: training, validation, and testing.
   - The Mistral-7B-Instruct-v0.3 language model and its tokenizer are loaded and prepared.

2. **Data Transformation with `TextToSQLDataset`:**
   - The `TextToSQLDataset` class is responsible for converting raw data into a format suitable for model training and evaluation.
   - For each data sample, the following transformations occur:
      - Tokenization: The input question and the SQL query (answer) are tokenized using the loaded tokenizer.
      - Dependency Parsing: The question is parsed using spaCy to extract the grammatical relationships between words, generating a dependency graph.
      - Dictionary Creation: A dictionary is created to store the tokenized input IDs, attention masks, labels (tokenized SQL query), and the dependency edges extracted from parsing.

3. **Batching and Shuffling with `DataLoader`:**
   - `DataLoader` takes the processed dataset from `TextToSQLDataset` and creates an iterable object for efficient batching.
   - Optionally, shuffling is applied to randomize the order of samples within each epoch during training.

4. **Forward Pass through `GraphModel`:**
   - **Mistral Encoder:** The tokenized input IDs and attention masks are fed into the Mistral model's encoder to obtain contextualized token embeddings.
   - **GATv2 Layer:** The GATv2 layer (Graph Attention Network) takes the token embeddings and the dependency edges as input. It applies graph attention mechanisms to incorporate the structural information from the dependency graph into the token representations.
   - **Pooling:** The node representations (output of GATv2) are aggregated using a pooling operation (e.g., mean pooling) to obtain a fixed-size representation of the entire input sequence.
   - **LM Head:** The pooled representation is passed through a linear layer, which produces logits – unnormalized probabilities for each token in the vocabulary.
   - **Loss Calculation:** During training, if labels (correct SQL queries) are available, the cross-entropy loss is calculated between the predicted logits and the true labels. This loss guides the optimization of the model's parameters.

5. **Model Optimization with PEFT (LoRA):**
   - The `GraphModel` is wrapped with PEFT's LoRA (Low-Rank Adaptation) configuration to enable parameter-efficient fine-tuning.
   - During training, only the parameters of the GATv2 layer and the LM head are updated, while the rest of the model parameters remain frozen.
   - The Hugging Face Trainer manages the training process, iterating over the dataset, computing gradients, and updating the model's parameters based on the calculated loss.

6. **Evaluation:**
   - After (or during) training, the model is evaluated on the validation and test sets.
   - The `compute_metrics` function decodes the predicted logits and labels back into text and assesses the model's performance using two metrics:
      - Semantic Similarity: This metric measures how semantically close the predicted SQL query is to the reference SQL query using SentenceTransformer embeddings.
      - Exact Match: This metric checks if the predicted SQL query matches the reference SQL query exactly.

7. **Output:**
    - The final output is the evaluation results, including semantic similarity and exact match scores, which provide insights into the model's ability to generate accurate SQL queries from natural language questions.

Let me know if you have any further questions or would like clarification on any of the steps!


You're absolutely right! I apologize for the error in the previous diagram. The arrow direction was indeed incorrect.

Here's the corrected graphical representation of the dataflow, with the arrow pointing *into* the `Trainer`:

```
+-------------------+          +----------------------+
| Dataset Loading   |          | TextToSQLDataset     |
|  - sql-create...  | -------> | - Tokenization       |
|  - Split: train...|          | - Dependency Parsing |
+-------------------+          | - Dict Creation      |
                               +----------------------+
                                         |
                                         v
                         +---------------------------+
                         | DataLoader                |
                         | - Batches, Shuffling (opt)|
                         +---------------------------+
                                         |
                                         v
                         +---------------------------+
                         | GraphModel                |
                         | - Mistral Encoder         |
                         | - GATv2 Layer             |
                         | - Pooling                 |
                         | - LM Head                 |
                         | - (Loss Calculation)      |
                         +---------------------------+
                                         |
                                         v
            +--------------+          +---------+         +-----------------+
            | PEFT (LoRA)  | -------> | Trainer | <------ |Evaluation       |
            +--------------+          |         |         |(compute_metrics)|
                                      +---------+         +-----------------+
                                         |
                                         v
                                   +-----------------------+
                                   | - Semantic Similarity |
                                   | - Exact Match         |
                                   +-----------------------+
```

The revised dataflow now accurately shows that the evaluation metrics (semantic similarity and exact match) calculated by the `compute_metrics` function are used by the `Trainer` to assess the model's performance and make decisions during training (e.g., early stopping).

Thank you for pointing out the error! I strive to be as accurate as possible in my responses.


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    BitsAndBytesConfig, Trainer, TrainingArguments,
    set_seed,
    EarlyStoppingCallback,
    DataCollatorForSeq2Seq
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    PeftConfig,
    PrefixTuningConfig,
    PromptEncoderConfig,
    TaskType
)


from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from torch_geometric.nn import GAT
import spacy
import numpy as np
import torch.nn as nn
import evaluate


# Suppress warnings
import warnings
warnings.simplefilter('ignore')

warnings.filterwarnings("ignore", message="The installed version of bitsandbytes was compiled without GPU support.")


# Load spaCy English model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    # Download if not already downloaded
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

# 1. Load and Prepare Data
dataset = load_dataset("b-mc2/sql-create-context")["train"].shuffle(seed=42)

# Manually define splits
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

#train_dataset = dataset.select(range(train_size))
#val_dataset = dataset.select(range(train_size, train_size + val_size))
test_dataset = dataset.select(range(train_size + val_size, len(dataset)))

# Optionally, load augmented data (if you have it)
# train_dataset = load_dataset("json", data_files="your_augmented_dataset.json", split="train")

# 2. Mistral Model and Tokenizer
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Model Configuration
config = AutoModelForCausalLM.from_pretrained(model_id).config
config.output_hidden_states = True
config.use_cache = False
#config.torch_dtype = torch.float32
config.torch_dtype = torch.bfloat16

# Load Model with Quantization
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #attn_implementation="flash_attention_2",  # Optimization
    torch_dtype=torch.bfloat16,
    #quantization_config=bnb_config,
    config=config
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. PyTorch Datasets

import torch
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
import spacy

# Load the spaCy English model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        text = item['messages'][1]['content']
        target_text = item['messages'][2]['content']

        # 1. Tokenization (with padding and truncation)
        tokenized_input = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=1024,  # Increase as needed
            return_tensors="pt"
        )
        tokenized_target = self.tokenizer(
            target_text,
            truncation=True,
            padding="max_length",
            max_length=1024,  # Increase as needed
            return_tensors="pt"
        )

        # Flatten lists (if needed)
        tokenized_input = {k: v.squeeze(0) for k, v in tokenized_input.items()}
        tokenized_target = {k: v.squeeze(0) for k, v in tokenized_target.items()}

        # 2. Dependency Parsing for Edge Extraction
        doc = nlp(text)
        edges = []
        for token in doc:
            head_i = token.head.i
            if 0 <= head_i < len(doc) and token.dep_ != "ROOT" and token.i != head_i:
                edges.append([token.i, head_i])

        # Edge Index Extraction and Validation
        edges = item.get("edges", edges)  # If "edges" is already present in data, use that

        if not edges:  # Handle empty graphs
            num_nodes = len(tokenized_input["input_ids"])
            edges = [[i, i] for i in range(num_nodes)]  # Self-loops for isolated nodes
        else:
            max_index = len(tokenized_input["input_ids"]) - 1
            edges = [(src, tgt) for src, tgt in edges if 0 <= src <= max_index and 0 <= tgt <= max_index]

        # Create edge index tensor
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        # Convert everything to tensors BEFORE padding/truncation
        input_ids = tokenized_input["input_ids"].clone().detach()
        attention_mask = tokenized_input["attention_mask"].clone().detach()
        labels = tokenized_target["input_ids"].clone().detach()

        # Handle potentially empty target sequences
        if len(labels) == 0:
            labels = torch.tensor([self.tokenizer.pad_token_id], dtype=torch.long)  # Create a single-element tensor with pad token

        # Padding and Truncation for consistent input shapes
        max_length = 1024  # Adjust if needed

        # Ensure that ALL tensors are truncated/padded to the SAME max_length
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]

        # Add padding if necessary
        if len(input_ids) < max_length:
            pad_length = max_length - len(input_ids)
            pad_tensor = torch.full((pad_length,), self.tokenizer.pad_token_id)
            input_ids = torch.cat((input_ids, pad_tensor))
            attention_mask = torch.cat((attention_mask, torch.zeros(pad_length, dtype=torch.long)))

        if len(labels) < max_length:
            pad_length = max_length - len(labels)
            labels = torch.cat((labels, torch.full((pad_length,), -100)))  # Pad labels with -100

        # (Optional) Print statements for debugging
        # print("\n")
        # print("Original text:", text)
        # print("Target text:", target_text)
        # print("Tokenized input IDs:", tokenized_input["input_ids"])
        # print("Tokenized target IDs:", tokenized_target["input_ids"])
        # print("Labels:", labels)
        # print("Edge index:", edge_index)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "edges": edge_index,
        }

train_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/train_dataset.json", split="train")
val_dataset   = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/test_dataset.json", split="train")


#Reduce train_dataset size for POC
POC_sample=10

import numpy as np
#train_dataset = train_dataset.select(range(100))
train_dataset = train_dataset.select(np.random.choice(len(train_dataset), POC_sample, replace=False))



train_dataset = TextToSQLDataset(train_dataset, tokenizer)
val_dataset = TextToSQLDataset(val_dataset, tokenizer)
test_dataset = TextToSQLDataset(test_dataset, tokenizer)

# Create DataLoader (no collate_fn needed)
#train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 4. GAT Layer and GraphModel
class GATLayer(torch.nn.Module):
    def __init__(self, in_features, out_features, num_heads=8, num_layers=1):
        super(GATLayer, self).__init__()
        self.gat = GAT(in_channels=in_features, hidden_channels=out_features, heads=num_heads,
                        concat=False, num_layers=num_layers)

    def forward(self, x, edge_index):
        return self.gat(x, edge_index)

    # Make the internal linear layers accessible
    def get_lora_target_modules(self):
        # Access the linear layers within the GAT convolutions
        return [module for module in self.gat.modules() if isinstance(module, torch.nn.Linear)]


#  GraphModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv

import torch
import torch.nn as nn
from torch_geometric.nn import GATv2Conv


#from transformers import CausalLMOutputWithPast

from dataclasses import dataclass
from typing import Optional, Tuple

@dataclass
class MyCausalLMOutputWithPast:
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


class GraphModel(nn.Module):
    def __init__(self, encoder, tokenizer):
        super(GraphModel, self).__init__()
        self.encoder = encoder
        self.config = encoder.model.config
        self.gatv2 = GATv2Conv(
            in_channels=self.config.hidden_size,
            out_channels=self.config.hidden_size,
            heads=8,
            concat=False,
        )

        # Max Pooling
        self.pool = lambda x, batch: torch.max(x, dim=0, keepdim=True)[0]

        # Additional Feedforward Layer
        self.ffn = nn.Sequential(
            nn.Linear(self.config.hidden_size, self.config.hidden_size * 2),
            nn.ReLU(),
            nn.Linear(self.config.hidden_size * 2, self.config.hidden_size),
        )

        self.lm_head = nn.Linear(self.config.hidden_size, self.config.vocab_size)
        self.tokenizer = tokenizer



    def forward(self, input_ids, attention_mask, edges, labels=None, inputs_embeds=None, output_attentions=False, output_hidden_states=False, return_dict=False):

        # Print vocabulary sizes
        #print('\n')
        #print("Mistral Model Vocab Size:", self.encoder.config.vocab_size)
        #print("Tokenizer Vocab Size:", self.tokenizer.vocab_size)
        #print('\n')


        # 1. Token Embeddings (Encoder)
        if input_ids is not None:
            encoder_outputs = self.encoder(
                input_ids.to(self.encoder.model.device),
                attention_mask=attention_mask.to(self.encoder.model.device),
                output_hidden_states=True
            )
            embeddings = encoder_outputs.hidden_states[-1]
        elif inputs_embeds is not None:
            embeddings = inputs_embeds
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        # Ensure correct shape for GATv2Conv
        if embeddings.dim() > 2:
            embeddings = embeddings.view(-1, embeddings.shape[-1])

        # 2. Edge Index Creation (Batched, with Enhanced Error Handling)
        edge_index = []
        node_offset = 0
        for i, graph_edges in enumerate(edges):
            if graph_edges.numel() == 0:
                num_nodes = input_ids.size(1)
                graph_edges = torch.arange(node_offset, node_offset + num_nodes, device=embeddings.device).repeat(2, 1)
            graph_edges += node_offset
            edge_index.append(graph_edges)
            node_offset += input_ids.size(1)
        edge_index = torch.cat(edge_index, dim=1)

        # 3. GATv2 Layer
        graph_out = self.gatv2(embeddings, edge_index)

        # 4. Pooling (using max pooling)
        batch = torch.arange(len(edges), device=graph_out.device).repeat_interleave(input_ids.size(1))
        pooled = self.pool(graph_out, batch).unsqueeze(1)

        # Additional Feedforward Layer
        pooled = self.ffn(pooled)

        # 5. LM Head
        logits = self.lm_head(pooled)

        # 6. Loss Calculation (if labels provided)
        loss = None
        if labels is not None:
            # Apply softmax to logits to get probabilities
            log_probs = F.log_softmax(logits, dim=-1)

            # Calculate loss
            loss_fct = nn.NLLLoss(ignore_index=-100)  # Use NLLLoss for log probabilities

            # Reshape log_probs to match target shape (remove the extra dimension)
            log_probs = log_probs.squeeze(1)
            # Reshape labels to match log_probs shape (batch_size, vocab_size)
            labels = labels[:, 0]
            # Calculate loss with the correctly shaped labels
            loss = loss_fct(log_probs, labels)
            # Print Iteration/Step and Loss
            print(f"\nIteration/Step: {trainer.state.global_step}, Loss: {loss.item()}\n")

        # 7. Return (Modified)
        if not return_dict:
            output = (logits,) + encoder_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return MyCausalLMOutputWithPast(
                loss=loss,
                logits=logits,
                past_key_values=encoder_outputs.past_key_values,
                hidden_states=encoder_outputs.hidden_states,
                attentions=encoder_outputs.attentions,
        )


    def prepare_inputs_for_generation(self, input_ids, edges, attention_mask=None, **kwargs):
        if isinstance(self, PeftModel):
            return self.base_model.prepare_inputs_for_generation(input_ids, edges, attention_mask, **kwargs)

        batch_size = input_ids.size(0)
        if batch_size > 1:
            batched_edges = []
            node_offset = 0
            for i in range(batch_size):
                graph_edges = edges[i]
                batched_edges.extend([(src + node_offset, dst + node_offset) for src, dst in graph_edges])
                node_offset += input_ids.size(1)
            edge_index = torch.tensor(batched_edges, dtype=torch.long).t().contiguous().to(input_ids.device)
        else:
            edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous().to(input_ids.device)

        model_inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "edges": edge_index,
            "past_key_values": kwargs.get("past_key_values", None),
        }
        return model_inputs

# END GraphModel

from peft import prepare_model_for_kbit_training

# Quantize Mistral (before creating GraphModel)
mistral_model = prepare_model_for_kbit_training(mistral_model, use_gradient_checkpointing=True)


# 5. Model Setup (Define model first)
model = GraphModel(mistral_model, tokenizer)  # Pass both mistral_model and tokenizer
#model.to(device)




# 6. PEFT Configuration (Use automatic module discovery)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    #target_modules="all-linear",
    task_type="CAUSAL_LM",
        target_modules=[
        "gat.gat.convs.0.lin_l",
        "gat.gat.convs.0.lin_r",
        "gat.gat.convs.1.lin_l",
        "gat.gat.convs.1.lin_r",
        "lm_head"
    ]
    #task_type=TaskType.CAUSAL_LM
)


# 7. Apply PEFT
model = get_peft_model(model, peft_config)
print('\n\n')
print('PEFT-Model')
model.print_trainable_parameters() # To see the trainable parameters
print('\n')

# Access the config of the encoder (Mistral model) within your GraphModel
model.encoder.config.use_cache = False
model.encoder.gradient_checkpointing_enable()  # Enable gradient checkpointing for memory optimization on the Mistral model
model.encoder.model.embed_tokens.requires_grad_(True)
torch.cuda.empty_cache()

# 8. Evaluation Metric (Semantic Similarity)
metric = evaluate.load("exact_match")
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute embeddings for predictions and references
    prediction_embeddings = sentence_transformer_model.encode(predictions, convert_to_tensor=True)
    reference_embeddings = sentence_transformer_model.encode(references, convert_to_tensor=True)

    # Calculate cosine similarities
    cosine_similarities = util.cos_sim(prediction_embeddings, reference_embeddings)
    similarities = torch.diag(cosine_similarities).cpu().numpy()  # Extract similarities for corresponding pairs

    # Return average similarity as the metric
    em = metric.compute(predictions=predictions, references=references)["exact_match"]
    return {"semantic_similarity": np.mean(similarities), "exact_match": em}


# 9. Training Arguments and Trainer
training_args = TrainingArguments(
    "graph-T2SQL",
    logging_dir="graph-T2SQL",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    push_to_hub=False,
    dataloader_pin_memory=False,
    load_best_model_at_end=True,
    #gradient_checkpointing=False,
    use_legacy_prediction_loop=False,
    metric_for_best_model="eval_semantic_similarity",
    report_to="tensorboard",
)


optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)


from torch.nn.utils.rnn import pad_sequence
from torch_geometric.data import Batch as GraphBatch  # Note the import
import torch
import torch_geometric.data
from torch.nn.utils.rnn import pad_sequence

class GraphDataCollatorForSeq2Seq:
    def __init__(self, tokenizer, model=None, label_pad_token_id=-100, pad_to_multiple_of=None):
        self.tokenizer = tokenizer
        self.model = model
        self.label_pad_token_id = label_pad_token_id
        self.pad_to_multiple_of = pad_to_multiple_of

    def __call__(self, features):
        # Separate standard features from graph edges
        # Extract labels before padding and handle potentially empty sequences
        labels = [feature["labels"] if feature["labels"].numel() > 0
                  else torch.tensor([self.label_pad_token_id], dtype=torch.long)
                  for feature in features]
        standard_features = [{k: v for k, v in feature.items() if k != "edges" and k != "labels"} for feature in features]
        edges = [feature["edges"] for feature in features]

        # Collate standard features (input_ids, attention_mask) using default collator
        collated_standard_features =  DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            model=self.model,
            label_pad_token_id=self.label_pad_token_id,
            pad_to_multiple_of=self.pad_to_multiple_of
        )(standard_features)

        # Pad input_ids and attention_mask
        input_ids = pad_sequence([f['input_ids'] for f in standard_features], batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence([f['attention_mask'] for f in standard_features], batch_first=True, padding_value=0)

         # Pad labels separately
        labels = pad_sequence(labels, batch_first=True, padding_value=self.label_pad_token_id)

        # Create batch for graph data
        graph_data_list = []
        for i in range(len(edges)):
            # Convert to PyTorch Geometric Data
            graph_data_list.append(torch_geometric.data.Data(
                x=collated_standard_features['input_ids'][i].unsqueeze(1),  # Node features (input_ids)
                edge_index=edges[i],                   # Edge index
                # Use num_edges for batch index to ensure correct batching in PyG
                batch=torch.tensor([i] * edges[i].size(1))
            ))
        batched_graph = GraphBatch.from_data_list(graph_data_list)  # Batch graphs

        # Keep edges as a list of tensors
        collated_features = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,  # Add the padded labels back
            "edges": edges  # Pass the list of edge index tensors directly
        }

        return collated_features


# 10a.  Data Collator
data_collator = GraphDataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)


from transformers import TrainerCallback

class LossLoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 100 == 0:  # Log every 100 steps (adjust as needed)
            print(f"Step {state.global_step} - Loss: {state.loss}")

print('\n\n')
print('GraphModel-Model')
print(model)
print('\n\n')

# 10. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,  # Use the custom collator
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)


def print_batch_shapes(args):
    # Access the current batch from the trainer state
    batch = args.state.batch

    # Print shapes for debugging
    print("Input IDs shape in batch:", batch["input_ids"].shape)
    print("Attention Mask shape in batch:", batch["attention_mask"].shape)
    print("Labels shape in batch:", batch["labels"].shape)
    print("Edges shape in batch:", [edge.shape for edge in batch["edges"]])

    # Print the actual values of 'edges' for a few samples
    for i in range(min(3, len(batch["edges"]))):
        print(f"Edges for sample {i}:", batch["edges"][i])



# Add the callback to the Trainer
trainer.add_callback(LossLoggingCallback())

# 11. Train the model
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
trainer.train()

print('\n\n')
# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')
print(f'Test Exact Match: {test_results["eval_exact_match"]:.4f}')

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]




GraphModel-Model
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraphModel(
      (encoder): MistralForCausalLM(
        (model): MistralModel(
          (embed_tokens): Embedding(32768, 4096)
          (layers): ModuleList(
            (0-31): 32 x MistralDecoderLayer(
              (self_attn): MistralSdpaAttention(
                (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
                (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
                (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
                (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
                (rotary_emb): MistralRotaryEmbedding()
              )
              (mlp): MistralMLP(
                (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
                (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
                (down_proj): Linear(in_features=14336, out_f

Epoch,Training Loss,Validation Loss



Iteration/Step: 1, Loss: 10.353950500488281


Iteration/Step: 1, Loss: 10.347931861877441


Iteration/Step: 2, Loss: 10.38474178314209


Iteration/Step: 2, Loss: 10.301795959472656


Iteration/Step: 3, Loss: 10.417816162109375


Iteration/Step: 3, Loss: 10.421366691589355


Iteration/Step: 4, Loss: 10.441377639770508


Iteration/Step: 4, Loss: 10.308401107788086



OutOfMemoryError: CUDA out of memory. Tried to allocate 1.01 GiB. GPU 

In [ ]:

# Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  # Ensure tensors divisible by 8 for optimized performance
)

# 10. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)


Let's analyze the information you've provided about your `GraphModel`:

**Structure Analysis:**

* **Encoder (Mistral-7B):** The backbone of your model is the Mistral-7B language model. This model is responsible for understanding the input text and generating meaningful representations (embeddings) for each token. The encoder has a substantial number of parameters (7.5 billion), indicating its large size and capacity to learn complex language patterns.

* **GAT Layer:**  You've incorporated a Graph Attention Network (GAT) layer to process the graph structure of your input data. This layer likely learns to weigh the importance of different nodes and edges based on their relationships, which can help capture additional information beyond just the raw text.

* **Pooling:** The `pool` operation is used to aggregate the representations from individual nodes into a single representation for the entire graph. The choice of pooling (e.g., mean pooling) depends on your specific task and how you want to combine the node-level information.

* **LM Head:** The language model head (`lm_head`) takes the final graph representation and predicts the next token in the sequence. It has a large number of parameters due to the vocabulary size (32,768).

**PEFT (Parameter-Efficient Fine-Tuning):**

- **Lora (Low-Rank Adaptation):** You are using LoRA, a technique that adds small adapter modules to the model to fine-tune it more efficiently. This allows you to train a smaller number of parameters while still achieving good performance.

- **Trainable Parameters:**  Only a small fraction (0.0078%) of the total parameters are trainable, thanks to LoRA. This drastically reduces the computational and memory requirements during training.

**Potential Challenges and Considerations:**

- **Graph Structure Quality:** The effectiveness of the GAT layer heavily relies on the quality of the graph structure you create from the input text. Dependency parsing is a good starting point, but you might want to explore other ways to construct the graph to better capture the relationships between words and phrases.

- **Overfitting with LoRA:** Even with LoRA, overfitting can be a concern, especially with a small dataset. Monitor your validation loss and consider techniques like early stopping or adding more training data (if possible).

- **Memory Usage:**  Graph models can be memory intensive, especially with large input sequences or large batch sizes. You might need to experiment with techniques like gradient checkpointing or gradient accumulation to manage memory usage.

- **Training Stability:** Training large language models can sometimes be unstable. Experiment with different learning rates, optimizers, and warm-up strategies to find the best settings for your model.

- **Evaluation Metrics:** Choose appropriate evaluation metrics that reflect the quality of the generated SQL queries. Consider using both automated metrics (e.g., BLEU score, accuracy) and human evaluation to assess the model's performance.


Let me know if you have any specific questions or concerns!


* Original Code - TRAINING ONLY


TO BE TEST IT WORK FOR trainer = SFTTrainer( ; from trl import SFTTrainer;

max_seq_length = 2048 --- max sequence length for model and packing of the dataset

 https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Mistral_7B_Instruct_v0_1_for_text_to_SQL_EVALDATA.ipynb



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 8. Evaluation Metric (Semantic Similarity)
metric = evaluate.load("exact_match")
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute embeddings for predictions and references
    prediction_embeddings = sentence_transformer_model.encode(predictions, convert_to_tensor=True)
    reference_embeddings = sentence_transformer_model.encode(references, convert_to_tensor=True)

    # Calculate cosine similarities
    cosine_similarities = util.cos_sim(prediction_embeddings, reference_embeddings)
    similarities = torch.diag(cosine_similarities).cpu().numpy()  # Extract similarities for corresponding pairs

    # Return average similarity as the metric
    em = metric.compute(predictions=predictions, references=references)["exact_match"]
    return {"semantic_similarity": np.mean(similarities), "exact_match": em}

# 9. Training Arguments and Trainer
training_args = TrainingArguments(
    "graph-T2SQL",
    logging_dir="graph-T2SQL",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    push_to_hub=False,
    dataloader_pin_memory=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_semantic_similarity",
    report_to="tensorboard",
)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8  # Ensure tensors divisible by 8 for optimized performance
)

print('\n\n')

# 10. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None)
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))


# 11. Train the model
print('\n\n')
trainer.train()
print('\n\n')

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')
print(f'Test Exact Match: {test_results["eval_exact_match"]:.4f}')

## Enhance - 1: Data Augmentation

In [ ]:
!pip install nlpaug -q

In [ ]:
from nlpaug.augmenter.word import SynonymAug, RandomWordAug
from nlpaug.augmenter.sentence import RandomInsertionAug

def augment_data(dataset):
    augmented_data = []

    synonym_aug = SynonymAug(aug_src='wordnet')
    insert_aug = RandomInsertionAug(aug_p=0.1)  # Insert with 10% probability
    delete_aug = RandomWordAug(action="delete", aug_p=0.05)  # Delete with 5% probability

    for item in dataset:
        question = item['question']
        answer = item['answer']

        # Create augmented examples
        augmented_questions = [
            synonym_aug.augment(question),
            insert_aug.augment(question),
            delete_aug.augment(question)
        ]

        # Add original and augmented examples to the dataset
        for aug_question in augmented_questions:
            augmented_data.append({'question': aug_question, 'answer': answer})

    return augmented_data

# Augment the training dataset
train_dataset_augmented = augment_data(train_dataset)

## Enhance - 2: Alternative Architecture: Graph Transformer Network (GTN)

In [ ]:
!pip install torch_geometric -q

In [ ]:
from torch_geometric.nn import GTConv

class GTNModel(torch.nn.Module):
    def __init__(self, encoder):
        super(GTNModel, self).__init__()
        self.encoder = encoder
        self.gtn = GTConv(4096, 4096, num_layers=2)  # 2 GTN layers
        self.pool = lambda x, batch: torch.mean(x, dim=0, keepdim=True)
        self.lm_head = torch.nn.Linear(4096, tokenizer.vocab_size)

    def forward(self, **inputs):
        x = inputs['input_ids'].to(mistral_model.device)
        edge_index = inputs['edge_index'].to(mistral_model.device)
        embeddings = self.encoder(x).last_hidden_state.cpu()
        gtn_out = self.gtn(embeddings, edge_index.cpu())
        pooled = self.pool(gtn_out, inputs.get('batch', torch.zeros(gtn_out.size(0)).long()))
        out = self.lm_head(pooled.to(mistral_model.device))
        return {"logits": out}

# Replace the model with the GTNModel
model = GTNModel(mistral_model)

## Enhance - 3: Integration Enhance - 1 and Enhance - 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
)
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import spacy
import numpy as np
from torch_geometric.nn import GTConv  # For GTN
from trl import setup_chat_format
from nlpaug.augmenter.word import SynonymAug, RandomWordAug
from nlpaug.augmenter.sentence import RandomInsertionAug

# ... (rest of the imports and model/tokenizer loading from the original code)

# Data Augmentation Function
def augment_data(dataset):
    augmented_data = []

    synonym_aug = SynonymAug(aug_src='wordnet')
    insert_aug = RandomInsertionAug(aug_p=0.1)  # Insert with 10% probability
    delete_aug = RandomWordAug(action="delete", aug_p=0.05)  # Delete with 5% probability

    for item in dataset:
        question = item['question']
        answer = item['answer']

        # Create augmented examples
        augmented_questions = [
            synonym_aug.augment(question),
            insert_aug.augment(question),
            delete_aug.augment(question)
        ]

        # Add original and augmented examples to the dataset
        for aug_question in augmented_questions:
            augmented_data.append({'question': aug_question, 'answer': answer})

    return augmented_data

# TextToSQLDataset (unchanged from the original code)
class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        #text = item['question'] + " " + item['context']
        text = item['question']
        target_text = item['answer']

        tokenized_input = self.tokenizer(text, truncation=True, padding=True, return_tensors="pt")
        tokenized_target = self.tokenizer(target_text, truncation=True, padding=True, return_tensors="pt")

        # Dependency Parsing for Edge Index
        doc = nlp(text)
        edges = []
        for token in doc:
            if token.i < len(doc) - 1:
                edges.append([token.i, token.head.i])
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        return {
            'input_ids': tokenized_input['input_ids'].flatten(),
            'attention_mask': tokenized_input['attention_mask'].flatten(),
            'labels': tokenized_target['input_ids'].flatten(),
            'edge_index': edge_index,
        }

# GTN Model WITH GTConv
from torch_geometric.nn import GTConv

class GTNModel(torch.nn.Module):
    def __init__(self, encoder):
        super(GTNModel, self).__init__()
        self.encoder = encoder
        self.gtn = GTConv(4096, 4096, num_layers=2)  # 2 GTN layers
        self.pool = lambda x, batch: torch.mean(x, dim=0, keepdim=True)
        self.lm_head = torch.nn.Linear(4096, tokenizer.vocab_size)

    def forward(self, **inputs):
        x = inputs['input_ids'].to(mistral_model.device)
        edge_index = inputs['edge_index'].to(mistral_model.device)
        embeddings = self.encoder(x).last_hidden_state.cpu()

        gtn_out = self.gtn(embeddings, edge_index.cpu())
        pooled = self.pool(gtn_out, inputs.get('batch', torch.zeros(gtn_out.size(0)).long()))
        out = self.lm_head(pooled.to(mistral_model.device))
        return {"logits": out}

# Replace the model with the GTNModel
model = GTNModel(mistral_model)
model = get_peft_model(model, peft_config)
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

# 7. Evaluation Metric (Semantic Similarity)
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute embeddings for predictions and references
    prediction_embeddings = sentence_transformer_model.encode(predictions, convert_to_tensor=True)
    reference_embeddings = sentence_transformer_model.encode(references, convert_to_tensor=True)

    # Calculate cosine similarities
    cosine_similarities = util.cos_sim(prediction_embeddings, reference_embeddings)
    similarities = torch.diag(cosine_similarities).cpu().numpy()  # Extract similarities for corresponding pairs

    # Return average similarity as the metric
    return {"semantic_similarity": np.mean(similarities)}


# Augment the training dataset
train_dataset_augmented = augment_data(train_dataset)

# Create datasets (using augmented training data)
train_dataset = TextToSQLDataset(train_dataset_augmented, tokenizer)
val_dataset = TextToSQLDataset(val_dataset, tokenizer)
test_dataset = TextToSQLDataset(test_dataset, tokenizer)

# Use the GTNModel
model = GTNModel(mistral_model)

model = get_peft_model(model, peft_config)
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

torch.cuda.empty_cache()

# 10. Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator=lambda x: x,
    compute_metrics = compute_metrics,
    optimizers=(optimizer, None)
)

from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

# 11. Train the model
trainer.train()

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')

## Enhance - 4: Regularization Techniques

In [ ]:
# ... (other imports)
from torch_geometric.nn import GTConv

# ... (other classes and functions)

# GTN Model with Dropout
class GTNModel(torch.nn.Module):
    def __init__(self, encoder, dropout_prob=0.1):  # Add dropout probability
        super(GTNModel, self).__init__()
        self.encoder = encoder
        self.gtn = GTConv(4096, 4096, num_layers=2)
        self.dropout = nn.Dropout(dropout_prob)  # Dropout layer
        self.pool = lambda x, batch: torch.mean(x, dim=0, keepdim=True)
        self.lm_head = torch.nn.Linear(4096, tokenizer.vocab_size)

    def forward(self, **inputs):
        # ... (rest of the forward method)
        x = inputs['input_ids'].to(mistral_model.device)
        edge_index = inputs['edge_index'].to(mistral_model.device)
        embeddings = self.encoder(x).last_hidden_state.cpu()


        gtn_out = self.gtn(embeddings, edge_index.cpu())
        gtn_out = self.dropout(gtn_out)  # Apply dropout after GTN

        pooled = self.pool(gtn_out, inputs.get('batch', torch.zeros(gtn_out.size(0)).long()))

        out = self.lm_head(pooled.to(mistral_model.device))
        return {"logits": out}

# ... (rest of the code)

# Create the model with dropout
model = GTNModel(mistral_model, dropout_prob=0.1)  # Set dropout probability
model = get_peft_model(model, peft_config)


model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

torch.cuda.empty_cache()

# 10. Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator=lambda x: x,
    compute_metrics = compute_metrics,
    optimizers=(optimizer, None)
)

from transformers import EarlyStoppingCallback
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))

# 11. Train the model
trainer.train()

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')


## Enhance - 5:



* MultiTaskDataset: This class combines multiple datasets and assigns task IDs to each sample.

* MultiTaskModel: This class extends the model to include a separate head for the classification task.

* Task-Specific Forward Pass: The forward method now takes a task_id input and uses the appropriate head based on the task.

* Training Loop: The training loop needs to be modified to handle the multi-task dataset and compute losses for both tasks.

In [ ]:
class TextToSQLDataset(Dataset):
    def __init__(self, dataset, tokenizer, task_id=0):  # Add task_id argument
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.task_id = task_id  # Store task ID

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        question = item['question'] if self.task_id == 0 else item['text']  # Use appropriate field
        answer = item['answer'] if self.task_id == 0 else item['label']  # Use appropriate field
        # ... (rest of the __getitem__ method for tokenization and graph construction)


        item = {'input_ids': input_ids, 'attention_mask': attention_mask,
                'edge_index': edge_index, 'task_id': self.task_id}  # Add task_id
        return item


In [ ]:
# Create the multi-task model with dropout
model = MultiTaskModel(mistral_model, dropout_prob=0.1, num_classes=num_classification_classes)
model = get_peft_model(model, peft_config)

In [ ]:
# ... (other parts of the code)

train_dataset_sql = TextToSQLDataset(train_dataset_augmented, tokenizer, task_id=0)
train_dataset_classification = TextToSQLDataset(dataset_classification, tokenizer, task_id=1)

# Combine datasets (consider balancing them if needed)
train_dataset = torch.utils.data.ConcatDataset([train_dataset_sql, train_dataset_classification])

# ... (rest of the training and evaluation code, modified for multi-task)


In the provided code, the new train_dataset brings several enhancements compared to the original version:

Data Augmentation: The most significant change is the incorporation of data augmentation. The original train_dataset is passed through the augment_data function, which creates additional training examples by applying techniques like synonym replacement, random word insertion, and random word deletion. This augmented dataset is then used to create the new train_dataset.

Benefit: Data augmentation helps the model generalize better to different phrasing and vocabulary variations in natural language input, leading to improved performance on unseen examples.
Multi-Task Learning: The new train_dataset is designed to support multi-task learning. It combines examples from both the original text-to-SQL task and an additional classification task. Each example in the dataset is associated with a task_id to indicate which task it belongs to.

Benefit: Multi-task learning allows the model to learn shared representations between related tasks, potentially improving performance and generalization on both tasks.
Dynamic Padding: While not explicitly mentioned, the code likely uses dynamic padding when creating the train_dataset. This means that each example is padded to the length of the longest sequence in its batch, rather than using a fixed maximum length.

Benefit: Dynamic padding reduces unnecessary padding tokens, which can speed up training and potentially improve model performance.

Sources and related content


## Enhance - 6: Custom Loss Function

In [ ]:
from transformers import EarlyStoppingCallback
from torch.nn import CrossEntropyLoss, MSELoss


# Custom Loss Function (integrated)
def compute_loss(model, inputs, return_outputs=False):
    labels = inputs.pop("labels")
    task_ids = inputs.pop("task_id")

    outputs = model(**inputs)

    sql_logits = outputs.logits[task_ids == 0]  # SQL task logits
    sql_labels = labels[task_ids == 0]          # SQL task labels
    sql_loss = CrossEntropyLoss()(sql_logits, sql_labels)

    classification_logits = outputs.logits[task_ids == 1]  # Classification task logits
    classification_labels = labels[task_ids == 1]          # Classification task labels
    classification_loss = MSELoss()(classification_logits, classification_labels)

    # Combine Losses (adjust weights as needed)
    loss = 0.5 * sql_loss + 0.5 * classification_loss

    return (loss, outputs) if return_outputs else loss

# 10. Create Trainer instance (with the custom loss)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=lambda x: x,
    compute_metrics=compute_metrics,
    compute_loss=compute_loss,  # Pass the custom loss function here
    optimizers=(optimizer, None)
)

# 11. Train the model
trainer.train()

# 12. Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(f'Test Semantic Similarity: {test_results["eval_semantic_similarity"]:.4f}')